In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

In [2]:
# SETUP

G_INPUT_SIZE = 1
D_INPUT_SIZE = 1
D_OUTPUT_SIZE = 1
HIDDEN_SIZE = 11

BATCH_SIZE = 100
LEARNING_RATE = 0.01
EPOCHS = 10 ** 6

In [3]:
# INIT

def z_sample(shape):
    return np.random.uniform(0, 1, shape)


def x_sample(shape):
    return np.random.normal(-1, 1, shape)


def generator(z):
    hidden = fully_connected(z, HIDDEN_SIZE, activation_fn=tf.sigmoid)
    output = fully_connected(hidden, D_INPUT_SIZE, activation_fn=tf.sigmoid)
    return output


def discriminator(x):
    hidden = fully_connected(x, HIDDEN_SIZE, activation_fn=tf.sigmoid)
    output = fully_connected(hidden, D_OUTPUT_SIZE, activation_fn=tf.sigmoid)
    return output

In [ ]:
with tf.device('/gpu:0'):
    with tf.variable_scope('G'):
        z = tf.placeholder(tf.float32, shape=(None, G_INPUT_SIZE))
        G = generator(z)

    with tf.variable_scope('D') as scope:
        x = tf.placeholder(tf.float32, shape=(None, D_INPUT_SIZE))
        D1 = discriminator(x)
        D2 = discriminator(G)

        loss_d = tf.reduce_mean(-tf.log(D1) - tf.log(1 - D2))
        loss_g = tf.reduce_mean(-tf.log(D2))

        trainable_vars = tf.trainable_variables()
        d_params = [v for v in trainable_vars if v.name.startswith('D/')]
        g_params = [v for v in trainable_vars if v.name.startswith('G/')]
        print("D params count: {}".format(len(d_params)))
        print("G params count: {}".format(len(g_params)))

        opt_d = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss_d)
        opt_g = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss_g)
        
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as session:
    session.run(tf.initialize_all_variables())

    for step in range(EPOCHS):
        zs = z_sample([BATCH_SIZE, G_INPUT_SIZE])
        loss_gs, _ = session.run([loss_g, opt_g], feed_dict={z: zs})

        xs = x_sample([BATCH_SIZE, D_INPUT_SIZE])
        zs = z_sample([BATCH_SIZE, G_INPUT_SIZE])
        loss_ds, _ = session.run([loss_d, opt_d], feed_dict={x: xs, z: zs})

        if step % 10000 == 0:
            print('Epoch {}: loss_d={} loss_g={}'.format(step, loss_ds, loss_gs))

D params count: 8
G params count: 4
Epoch 0: loss_d=2.024170160293579 loss_g=0.2465161383152008
Epoch 10000: loss_d=0.700162947177887 loss_g=0.6980217099189758
Epoch 20000: loss_d=0.698911726474762 loss_g=0.6978857517242432
Epoch 30000: loss_d=0.6983233094215393 loss_g=0.697760820388794
Epoch 40000: loss_d=0.6980656385421753 loss_g=0.697624146938324
Epoch 50000: loss_d=0.6978524923324585 loss_g=0.6975095868110657
Epoch 60000: loss_d=0.6976476907730103 loss_g=0.6973869204521179
Epoch 70000: loss_d=0.6974903345108032 loss_g=0.6972706317901611
Epoch 80000: loss_d=0.6973512172698975 loss_g=0.6971679925918579
Epoch 90000: loss_d=0.6972470879554749 loss_g=0.6970666646957397
Epoch 100000: loss_d=0.6971054673194885 loss_g=0.696976900100708
Epoch 110000: loss_d=0.6969719529151917 loss_g=0.6968749761581421
Epoch 120000: loss_d=0.6969310641288757 loss_g=0.6967953443527222
Epoch 130000: loss_d=0.6967716813087463 loss_g=0.6967225670814514
Epoch 140000: loss_d=0.6966955661773682 loss_g=0.69660067558